In [1]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

In [2]:
# data = pd.read_csv("More_player_stat.csv")
data = pd.read_csv("scrapped_player_stats-18th-Aug.csv")

In [3]:
# Assuming 'data' is your DataFrame
# Extract 3 letter country code 
data['nation'] = data['nation'].str.extract(r'([A-Z]{3})')

In [4]:
# List of columns with float64 dtype
float64_columns = ['age', 'min', '90s', 'gls', 'ast', 'g+a', 'g-pk', 'pk', 'pkatt', 'crdy', 'crdr', 'xg', 'npxg', 'xag', 'npxg+xag']

# Convert selected columns to float32 dtype
data[float64_columns] = data[float64_columns].fillna(0).astype('int')

In [186]:
# # Filter for Manchester United players in the 2022 season
# man_utd_players = data[data['Team'] == "Manchester United"][data['Season'] == 2022]

# # Print the filtered DataFrame
# print(man_utd_players)


# Find Bruno Fernandes' age
bruno_fernandes_age = data[data['player'] == "Bruno Fernandes"]['age'].values[0]

# Print Bruno Fernandes' age
print(bruno_fernandes_age)

27


# Base code if all fail's return to this please

In [ ]:
app = dash.Dash()

app.layout = html.Div([
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='team-dropdown', 
            options=[{'label': t, 'value': t} for t in data['Team'].unique()],
            value=data['Team'].iloc[0]
        )),
        dbc.Col(dcc.Dropdown(
            id='season-dropdown',
            options=[{'label': s, 'value': s} for s in data['Season'].unique()],
            value=data['Season'].iloc[0] 
        )),
        dbc.Col(dcc.Dropdown(
            id='player-dropdown',
            options=[{'label': p, 'value': p} for p in data['player'].unique()],
            value=data['player'].iloc[0]
        ))
    ], ),
    html.Div(id='player-summary')
])

@app.callback(
    Output('player-dropdown', 'value'),
    [Input('team-dropdown', 'value'),
     Input('season-dropdown', 'value')]
)

def set_player_value(selected_team, selected_season):   
    if selected_team is None or selected_season is None:
        return None
    
    selected = data[(data['Team'] == selected_team) &
                   (data['Season'] == selected_season)]
                   
    return selected['player'].iloc[0]

@app.callback(
    Output('player-dropdown', 'options'),
    [Input('team-dropdown', 'value'),
     Input('season-dropdown', 'value')]
)  
def update_players(selected_team, selected_season):

    if selected_team is None:
        return [{'label': p, 'value': p} for p in data['player'].unique()]
    else:
        selected = data[(data['Team'] == selected_team) &
                        (data['Season'] == selected_season)]
        return [{'label': p, 'value': p} for p in selected['player'].unique()]

@app.callback(
    Output('player-summary', 'children'),
    [Input('player-dropdown', 'value'),
     Input('season-dropdown', 'value')]
)

def update_summary(selected_player, selected_season):
   

    player_data = data[(data['player'] == selected_player) &  
                       (data['Season'] == selected_season)]

    age = player_data['age'].values[0]
    nation = player_data['nation'].iloc[0]
    position = player_data['pos'].sum()              
    goals = player_data['gls'].sum()
    assists = player_data['ast'].sum()
    matches = player_data['starts'].sum()
    minutes_played = player_data['min'].sum()
    non_penalty_goals = player_data['g-pk'].sum()
    red_cards = player_data['crdr'].sum()
    yellow_cards = player_data['crdy'].sum()
    
    summary = html.Div([
        html.P(f"{selected_player} in {selected_season}"),
        html.P(f"Age: {age}"),
        html.P(f"Goals: {goals}"),
        html.P(f"Nationality: {nation}"),
        html.P(f"Position: {position}"),
        html.P(f"Assists: {assists}"),
        html.P(f"Matches Started: {matches}"),
        html.P(f"Minutes Played: {minutes_played}"),
        html.P(f"Non penalty goals: {non_penalty_goals}"),
        html.P(f"Number of red cards: {red_cards}"),
        html.P(f"Number of yellow cards: {yellow_cards}")
    ])
    
    return summary

# def update_summary(selected_player, selected_season):
#     player_data = data[(data['player'] == selected_player) & (data['Season'] == selected_season)]

#     summary_items = [
#         html.P(f"{selected_player} in {selected_season}", style={'font-weight': 'bold', 'font-size': '18px'})
#     ]

#     fields = {
#         'age': 'Age',
#         'nation': 'Nationality',
#         'pos': 'Position',
#         'gls': 'Goals',
#         'ast': 'Assists',
#         'starts': 'Matches Started',
#         'min': 'Minutes Played',
#         'g-pk': 'Non Penalty Goals',
#         'crdr': 'Red Cards',
#         'crdy': 'Yellow Cards'
#     }

#     for field, label in fields.items():
#         if field in player_data:
#             summary_items.append(html.P(f"{label}: {player_data[field].sum()}"))

#     summary_content = html.Div(summary_items)

#     card = dbc.Card(
#         dbc.CardBody(summary_content),
#         style={'margin': '20px', 'box-shadow': '0 4px 8px 0 rgba(0,0,0,0.2)', 'border-radius': '10px'}
#     )

#     return card


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

## Secondary Experimental code which semi works, but needs more error handling guardrails

In [ ]:
app = dash.Dash()

app.layout = html.Div([
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='team-dropdown', 
            options=[{'label': t, 'value': t} for t in data['Team'].unique()],
            value=data['Team'].iloc[0]
        )),
        dbc.Col(dcc.Dropdown(
            id='season-dropdown',
            options=[{'label': s, 'value': s} for s in data['Season'].unique()],
            value=data['Season'].iloc[0] 
        )),
        dbc.Col(dcc.Dropdown(
            id='player-dropdown',
            options=[{'label': p, 'value': p} for p in data['player'].unique()],
            value=data['player'].iloc[0]
        )),
        dbc.Col(dcc.Dropdown(
            id='player-dropdown-2',
            options=[{'label': p, 'value': p} for p in data['player'].unique()],
            multi=False,  # Allow only one selection
            placeholder="Select Player 2"
        )),
       dbc.Col(dcc.Dropdown(
            id='team-dropdown-2', 
            options=[{'label': t, 'value': t} for t in data['Team'].unique()],
            value=data['Team'].iloc[0]
        )),
    ], ),
    html.Div(id='player-summary')
])


@app.callback(
    [Output('player-dropdown', 'options'),
     Output('player-dropdown', 'value'),
     Output('player-dropdown-2', 'options'),
     Output('player-dropdown-2', 'value')],
    [Input('team-dropdown', 'value'),
     Input('team-dropdown-2', 'value'),
     Input('season-dropdown', 'value')]
)
def update_players_and_value(selected_team_1, selected_team_2, selected_season):
    if selected_team_1 is None:
        return (
            [{'label': p, 'value': p} for p in data['player'].unique()],
            data['player'].iloc[0],
            [{'label': p, 'value': p} for p in data['player'].unique()],
            data['player'].iloc[0]
        )
    else:
        selected_1 = data[(data['Team'] == selected_team_1) & (data['Season'] == selected_season)]
        selected_2 = data[(data['Team'] == selected_team_2) & (data['Season'] == selected_season)]
        return (
            [{'label': p, 'value': p} for p in selected_1['player'].unique()],
            selected_1['player'].iloc[0],
            [{'label': p, 'value': p} for p in selected_2['player'].unique()],
            selected_2['player'].iloc[0]
        )

@app.callback(
    Output('player-summary', 'children'),
    [Input('player-dropdown', 'value'),
     Input('player-dropdown-2', 'value'),
     Input('season-dropdown', 'value')]
)

def update_summary(player_1, player_2, selected_season):
    player_data_1 = data[(data['player'] == player_1) & (data['Season'] == selected_season)]
    player_data_2 = data[(data['player'] == player_2) & (data['Season'] == selected_season)]

    summary_items = [
        html.P(f"{player_1} vs {player_2} in {selected_season}", style={'font-weight': 'bold', 'font-size': '18px'})
    ]

    fields = {
        'age': 'Age',
        'nation': 'Nationality',
        'pos': 'Position',
        'gls': 'Goals',
        'ast': 'Assists',
        'starts': 'Matches Started',
        'min': 'Minutes Played',
        'g-pk': 'Non Penalty Goals',
        'crdr': 'Red Cards',
        'crdy': 'Yellow Cards'
    }

    for field, label in fields.items():
        if field in player_data_1 and field in player_data_2:
            summary_items.append(html.P(f"{label}: {player_data_1[field].sum()} vs {player_data_2[field].sum()}"))

    summary_content = html.Div(summary_items)

    card = dbc.Card(
        dbc.CardBody(summary_content),
        style={'margin': '20px'}
    )

    return card



if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

# Offbeat experiment


In [154]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='team-dropdown', 
            options=[{'label': t, 'value': t} for t in data['Team'].unique()],
            value=data['Team'].iloc[0]
        )),
       dbc.Col(dcc.Dropdown(
            id='team-dropdown-2', 
            options=[{'label': t, 'value': t} for t in data['Team'].unique()],
            value=data['Team'].iloc[1]
        )),
        dbc.Col(dcc.Dropdown(
            id='season-dropdown',
            options=[{'label': s, 'value': s} for s in data['Season'].unique()],
            value=data['Season'].iloc[0] 
        )),
        dbc.Col(dcc.Dropdown(
            id='player-dropdown',
            options=[{'label': p, 'value': p} for p in data['player'].unique()],
            value=data['player'].iloc[0]
        )),
        dbc.Col(dcc.Dropdown(
            id='player-dropdown-2',
            options=[{'label': p, 'value': p} for p in data['player'].unique()],
            multi=False, 
            placeholder="Select Player 2"
        )),
    ], ),
    html.Div(id='player-summary')
])

@app.callback(
    [Output('player-dropdown', 'options'),
     Output('player-dropdown', 'value'),
     Output('player-dropdown-2', 'options'),
     Output('player-dropdown-2', 'value')],
    [Input('team-dropdown', 'value'),
     Input('team-dropdown-2', 'value'),
     Input('season-dropdown', 'value')]
)

def update_players_and_value(selected_team_1, selected_team_2, selected_season):
    if not selected_team_1 or not selected_team_2:
        return [], None, [], None

    selected_1 = data[(data['Team'] == selected_team_1) & (data['Season'] == selected_season)]
    selected_2 = data[(data['Team'] == selected_team_2) & (data['Season'] == selected_season)]

    player_options_1 = [{'label': p, 'value': p} for p in selected_1['player'].unique()]
    player_options_2 = [{'label': p, 'value': p} for p in selected_2['player'].unique()]
    
    return player_options_1, player_options_1[0]['value'], player_options_2, player_options_2[1]['value']


def generate_player_card(player, player_data, selected_season):
    summary_items = [
        html.P(f"{player} in {selected_season}", style={'font-weight': 'bold', 'font-size': '18px'})
    ]

    fields = {
        'age': 'Age',
        'nation': 'Nationality',
        'pos': 'Position',
        'gls': 'Goals',
        'ast': 'Assists',
        'starts': 'Matches Started',
        'min': 'Minutes Played',
        'g-pk': 'Non Penalty Goals',
        'crdr': 'Red Cards',
        'crdy': 'Yellow Cards'
    }

    for field, label in fields.items():
        if field in player_data:
            summary_items.append(html.P(f"{label}: {player_data[field].values[0]}"))

    summary_content = html.Div(summary_items)

    card = dbc.Card(
        dbc.CardBody(summary_content),
        style={'margin': '20px'}
    )

    return card


@app.callback(
    Output('player-summary', 'children'),
    [Input('player-dropdown', 'value'),
     Input('player-dropdown-2', 'value'),
     Input('season-dropdown', 'value')]
)

def update_summary(player_1, player_2, selected_season):
    if not player_1 or not player_2:
        return html.P("Please select players from both teams to compare.")

    player_data_1 = data[(data['player'] == player_1) & (data['Season'] == selected_season)]
    player_data_2 = data[(data['player'] == player_2) & (data['Season'] == selected_season)]

    player_1_card = generate_player_card(player_1, player_data_1, selected_season)
    player_2_card = generate_player_card(player_2, player_data_2, selected_season)

    cards_row = dbc.Row([
        dbc.Col(player_1_card, width=6),
        dbc.Col(player_2_card, width=6)
    ])

    return cards_row




if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


# Layout of the page being done

In [ ]:
import dash
import dash_bootstrap_components as dbc
from dash import html
from dash.dependencies import Input, Output

app = dash.Dash(__name__, title='Premier league', external_stylesheets=[dbc.themes.BOOTSTRAP])

# Sidebar styles
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "20%",
    "padding": "1rem 2rem",
    "background-color": "#f8f9fa",
}

# Content styles
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

# Sidebar
sidebar = html.Div(
    [
        dbc.Nav(
            [
                dbc.NavLink("Home Page", href="/", active="exact"),
                dbc.NavLink("Page 1", href="/page-1", active="exact"),
                dbc.NavLink("Page 2", href="/page-2", active="exact"),
                dbc.NavLink("Page 3", href="/page-3", active="exact"),
            ],
            vertical=True,
            pills=True,
        )
    ],
    style=SIDEBAR_STYLE,
)

app.layout = html.Div(
    [
        dcc.Location(id="url", pathname="/"),  
        sidebar,
        html.Div(id="page-content", style=CONTENT_STYLE),
    ]
)

@app.callback(Output("page-content", "children"), [Input("url", "pathname")])

def render_page_content(pathname):    
    if pathname == "/":
        return html.Div(
            [
                html.H1("Home Page"),
                html.P("An overview of this dashboard and Sports Analytics."),
            ]
        )
    elif pathname == "/page-1":
        return html.Div(
            [
                html.H1("Page 1 Content"),
                html.P("Welcome to Page 1 of the Premier League Dashboard."),
            ]
        )
    elif pathname == "/page-2":
        return html.Div(
            [
                html.H1("Page 2 Content"),
                html.P("Explore insights and data related to the Premier League."),
            ]
        )
    elif pathname == "/page-3":
        return html.Div(
            [
                html.H1("Page 3 Content"),
                html.P("Get in-depth statistics and visualizations."),
            ]
        )

    return html.Div(
        [
            html.H1("404 Page Not Found"),
            html.P("The requested page was not found."),
        ]
    )

if __name__ == "__main__":
    app.run_server(debug=True, use_reloader = True)

# Radar chart added

In [ ]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='team-dropdown', 
            options=[{'label': t, 'value': t} for t in data['Team'].unique()],
            value=data['Team'].iloc[0]
        )),
       dbc.Col(dcc.Dropdown(
            id='team-dropdown-2', 
            options=[{'label': t, 'value': t} for t in data['Team'].unique()],
            value=data['Team'].iloc[1]
        )),
        dbc.Col(dcc.Dropdown(
            id='season-dropdown',
            options=[{'label': s, 'value': s} for s in data['Season'].unique()],
            value=data['Season'].iloc[0] 
        )),
        dbc.Col(dcc.Dropdown(
            id='player-dropdown',
            options=[{'label': p, 'value': p} for p in data['player'].unique()],
            value=data['player'].iloc[0]
        )),
        dbc.Col(dcc.Dropdown(
            id='player-dropdown-2',
            options=[{'label': p, 'value': p} for p in data['player'].unique()],
            multi=False, 
            placeholder="Select Player 2"
        )),
    ], ),
    html.Div(id='player-summary'),
    dcc.Graph(id='radar-chart')
])

@app.callback(
    [Output('player-dropdown', 'options'),
     Output('player-dropdown', 'value'),
     Output('player-dropdown-2', 'options'),
     Output('player-dropdown-2', 'value')],
    [Input('team-dropdown', 'value'),
     Input('team-dropdown-2', 'value'),
     Input('season-dropdown', 'value')]
)

def update_players_and_value(selected_team_1, selected_team_2, selected_season):
    if not selected_team_1 or not selected_team_2:
        return [], None, [], None

    selected_1 = data[(data['Team'] == selected_team_1) & (data['Season'] == selected_season)]
    selected_2 = data[(data['Team'] == selected_team_2) & (data['Season'] == selected_season)]

    player_options_1 = [{'label': p, 'value': p} for p in selected_1['player'].unique()]
    player_options_2 = [{'label': p, 'value': p} for p in selected_2['player'].unique()]
    
    return player_options_1, player_options_1[0]['value'], player_options_2, player_options_2[1]['value']


def generate_player_card(player, player_data, selected_season):
    summary_items = [
        html.P(f"{player} in {selected_season}", style={'font-weight': 'bold', 'font-size': '18px'})
    ]

    fields = {
        'age': 'Age',
        'nation': 'Nationality',
        'pos': 'Position',
        'gls': 'Goals',
        'ast': 'Assists',
        'starts': 'Matches Started',
        'min': 'Minutes Played',
        'g-pk': 'Non Penalty Goals',
        'crdr': 'Red Cards',
        'crdy': 'Yellow Cards'
    }

    for field, label in fields.items():
        if field in player_data:
            summary_items.append(html.P(f"{label}: {player_data[field].values[0]}"))

    summary_content = html.Div(summary_items)

    card = dbc.Card(
        dbc.CardBody(summary_content),
        style={'margin': '20px'}
    )

    return card


@app.callback(
    Output('player-summary', 'children'),
    [Input('player-dropdown', 'value'),
     Input('player-dropdown-2', 'value'),
     Input('season-dropdown', 'value')]
)

def update_summary(player_1, player_2, selected_season):
    if not player_1 or not player_2:
        return html.P("Please select players from both teams to compare.")

    player_data_1 = data[(data['player'] == player_1) & (data['Season'] == selected_season)]
    player_data_2 = data[(data['player'] == player_2) & (data['Season'] == selected_season)]

    player_1_card = generate_player_card(player_1, player_data_1, selected_season)
    player_2_card = generate_player_card(player_2, player_data_2, selected_season)
    

    cards_row = dbc.Row([
        dbc.Col(player_1_card, width=6),
        dbc.Col(player_2_card, width=6)
    ])

    return cards_row



@app.callback(
    Output('radar-chart', 'figure'),
    [Input('player-dropdown', 'value'),
     Input('player-dropdown-2', 'value'),
     Input('season-dropdown', 'value')]
)
def update_radar_chart(player_1, player_2, selected_season):
    if not player_1 or not player_2:
        return go.Figure()

    player_data_1 = data[(data['player'] == player_1) & (data['Season'] == selected_season)]
    player_data_2 = data[(data['player'] == player_2) & (data['Season'] == selected_season)]

    # Extract relevant attributes for radar chart
    attributes = ['age','gls', 'ast', 'starts', 'min', 'prgc', 'prgp']

    
    values_1 = player_data_1[attributes].values[0]
    values_2 = player_data_2[attributes].values[0]
    
    maxxx =  range=[0, max(max(values_1), max(values_2))]

    # # Convert values to regular Python floats
    # values_1 = [float(value) for value in values_1]
    # values_2 = [float(value) for value in values_2]
    
    # values_2 = [player_data_2[attr].sum() for attr in attributes]
    
    print(values_1)
    print(values_2)
    print(maxxx)

    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=values_1,
        theta=attributes,
        fill='toself',
        name=player_1
    ))

    fig.add_trace(go.Scatterpolar(
        r=values_2,
        theta=attributes,
        fill='toself',
        name=player_2
    ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 120])
        ),
        showlegend=True
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
